In [1]:
#Author: Will Cross-Bermingham

import pandas as pd
import numpy as np
from datetime import timedelta, date
import plotly.express as px
from fredapi import Fred
import os
import shutil
from fpdf import FPDF

#Config is a user file stored in the parent directory
import sys
sys.path.insert(1, '../')
import config

# User will need a FRED API key, can sign up for one here https://fred.stlouisfed.org/docs/api/api_key.html
# replace with your own API key: fred = Fred(api_key='YOUR KEY HERE')
fred = Fred(api_key=config.api_key)

In [2]:
#Define the plot object class
#fredget function will create a list of plot objects 
#Plot object class has a built-in plotting method allowing the user to automatically plot the series

class plot_obj:
  def __init__(self, df, title, units, years_back, height = 400, width = 500):
    
    #Restrict the plot sample to the past x years 
    #Where x is given by the "years_back" argument
    end_date_plot = df.index.max()
    start_date_plot = end_date_plot - timedelta(days = (365*years_back))
    self.df = df.loc[start_date_plot:end_date_plot]
    
    self.title = title
    self.units = units
    self.height = height
    self.width = width
    
  
  def plot_clean(self):
    colnames = self.df.columns
    fig = px.line(self.df, x=self.df.index, y=colnames, title=self.title, template="plotly_white", width = self.width, height = self.height )

    fig.update_layout(
        
        font_family="Georgia",
        font_color="black",
        yaxis_title = self.units,
        xaxis_title = "Date"
    )
    fig.update_traces(connectgaps=True)
    fig.update_traces(line_color='#b30531', line_width=2)
    fig.update_layout(showlegend=False)
    return fig



In [3]:
#Define a function to retrieve series, store as objects, and plot data 
#Inputs: Define an end data for the data 
def fredget(varlist,start_date, end_date, plots=True,freq="",name_map = dict()):
    
    df_collect = pd.DataFrame()
    plot_obs = list()
    idx = 0
    figlist = []
    page_fig_list = []
    page_count = 0
    PLOT_DIR = 'plots'
    if plots == True:
        try:
            shutil.rmtree(PLOT_DIR)
            os.mkdir(PLOT_DIR)
        except FileNotFoundError:
            os.mkdir(PLOT_DIR)
        
    for var in varlist:
        #series_add = fred.get_series(var, observation_start=s_d, observation_end=e_d,frequency ='m')
        df_add = fred.get_series(var, observation_start = start_date, observation_end=end_date, frequency =freq).to_frame(name = var)
        info_add = fred.get_series_info(var).to_frame(name = var)
        
        if var in name_map:
            info_add.loc['title', var] = name_map[var]
        
        plot_obs.append(plot_obj(df_add, info_add.loc['title', var], info_add.loc['units', var],5))
        
        print(var)
        if plots == True:
            #df_add.plot(y=var, use_index=True, kind='line', title = info_add.loc['title', var], ylabel = info_add.loc['units', var] )
            fig = plot_obs[idx].plot_clean()
            fig.show()
            filename = f"{PLOT_DIR}/fig{idx}.png"
            fig.write_image(filename,scale=3, width=500, height=400)
            figlist.append(filename)

       
        idx = idx + 1 
        #Create the list of plot images which is passed to the pdf function
        if idx % 6 == 0 :
            page_fig_list.append(figlist)
            figlist = []
            page_count = page_count + 1
            
        if df_collect.empty == True:
            df_collect = df_add
            info_store = info_add
        else:
            df_collect = pd.merge(df_collect, df_add, left_index=True, right_index=True, how='outer')
            info_store = pd.merge(info_store, info_add, left_index=True, right_index=True, how='left')
            
    df_collect.sort_index()
    
    return df_collect,info_store,page_fig_list



In [4]:
#CODE BORROWED FROM THIS MEDIUM ARTICLE:
#Main changes to original code: set up report for six 4x5 aspect ratio charts
#LINK: https://towardsdatascience.com/how-to-create-pdf-reports-with-python-the-essential-guide-c08dd3ebf2ee

class PDF(FPDF):
    def __init__(self):
        super().__init__()
        self.WIDTH = 210
        self.HEIGHT = 297
        
    def header(self):
        # Custom logo and positioning
        # Create an `assets` folder and put any wide and short image inside
        # Name the image `logo.png`
        today = date.today()
        date_print = today.strftime("%b-%d-%Y")
        self.image('assets/logo.png', 10, 8, 50)
        self.set_font('Times', 'B', 12)
        self.cell(self.WIDTH - 80)
        self.cell(60, 16, f'Financial Markets Overview: {date_print}', 0, 0, 'R')
        self.ln(20)
        
    def footer(self):
        # Page numbers in the footer
        self.set_y(-15)
        self.set_font('Arial', 'I', 12)
        self.set_text_color(128)
        self.cell(0, 10, 'Page ' + str(self.page_no()), 0, 0, 'C')

    def page_body(self, images):
        # Determine how many plots there are per page and set positions
        # and margins accordingly
        
        try:
            self.image(images[0], 15, 25, 95)
            self.image(images[1], 15, self.WIDTH / 2 + 5, 95)
            self.image(images[2], 15, self.WIDTH / 2 + 90, 95)
            self.image(images[3], 106, 25, 95)
            self.image(images[4], 106, self.WIDTH / 2 + 5, 95)
            self.image(images[5], 106, self.WIDTH / 2 + 90, 95)
            
        except:
            print(f"not enough images to fill page: Only have {len(images)} out of 6")
            
    def print_page(self, images):
        # Generates the report
        self.add_page()
        self.page_body(images)

In [5]:

#Set Options Here:
start_d ='1982-01-01'
end_d=date.today()

# VERY IMPORTANT, FRED CODES MUST BE MULTIPLES OF SIX: 6,12,... 
FRED_list = ['T10Y2Y','WALCL','EFFR','SP500','MORTGAGE30US','BAMLH0A0HYM2','DEXUSEU','DEXUSUK','DTWEXBGS','DCOILBRENTEU','DEXJPUS','DEXCHUS']
Name_remap = {'T10Y2Y': '10-Year Minus 2-Year Treasury', 'WALCL': 'Reserve Balances: Total Assets'}

df,dfinfo,figlist = fredget(FRED_list,start_d,end_d,name_map = Name_remap)

#output the PDF 
pdf = PDF()
for elem in figlist:
    pdf.print_page(elem)
   
#Save the generated file    
pdf.output('MacroReport.pdf', 'F')


T10Y2Y


WALCL


EFFR


SP500


MORTGAGE30US


...

DTWEXBGS


DCOILBRENTEU


DEXJPUS


DEXCHUS


''